# Generating images with MX-Font model from a reference style
In this example we'll generate images with trained MX-Font model from a reference style.
If you want to generate multiple styles, please check using `eval.py` instead of using this example file (because it is much simpler to load the referece styles).

### 1. Loading packages
* First, load the packages used in this code.
* All of the packages are avilable in `pip`.

In [1]:
import json
from pathlib import Path
from PIL import Image

import torch
from sconf import Config
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

* These modules are defined in this repository.

In [2]:
from base.dataset import read_font, render
from base.utils import save_tensor_to_image, load_reference
from MX.models import Generator
from inference import infer_MX

### 2. Build model
* Build and load the trained model.
* `weight_path` : 
    * The location of the trained model weight.
* `n_experts` :
    * The number of 'experts' in the trained model.

In [3]:
########################################################
weight_path = "weights/kor/mxfont.pth"  # path to weight to infer
n_experts = 3
########################################################

cfg = Config("cfgs/MX/default.yaml")

g_kwargs = cfg.get('g_args', {})
gen = Generator(n_experts=n_experts, n_emb=2).cuda().eval()
weight = torch.load(weight_path)
if "generator_ema" in weight:
    weight = weight["generator_ema"]
gen.load_state_dict(weight)

<All keys matched successfully>

### 3. Load reference images.
* `ref_path`: 
    * The path of reference font or images.
    * If you are using a ttf file, set this to the location of the ttf file.
    * If you want to use rendered images, set this to the path to the directory which contains the reference images.
* `ref_chars`:
    * The characters of reference images.
    * If this is `None`, all the available images will be loaded.
* `extension`:
    * If you are using ttf files, set this to "ttf".
    * If you are using image files, set this to their extension(png, jpg, etc..).

In [7]:
########################################################
ref_path = "data_example/kor/png"
extension = "png"
ref_chars = "값넋닻볘츄퀭핥훟"
## Comment upper lines and uncomment lower lines to test with ttf files.
# extension = "ttf"
# ref_chars = "값같곬곶깎넋늪닫닭닻됩뗌략몃밟볘뺐뽈솩쐐앉않얘얾엌옳읊죡쮜춰츄퀭틔핀핥훟"
########################################################

ref_dict, load_img = load_reference(ref_path, extension, ref_chars)

### 4. Generate the images.
* `gen_chars`: The characters to generate.
* `save_dir`: Path to save the generated images.
* `source_path`: Path to the source.
* `source_ext`: Extension of the source file. If you are using image files, set this to the image's extension.
* `batch_size`: The number of images inferred at once.

In [8]:
########################################################
gen_chars = "좋은하루되세요"  # Characters to generate
save_dir = "./result/mx"  # Directory where you want to save generated images
source_path = "data/kor/source.ttf"  # Path to the font file to render the source images
source_ext = "ttf"
batch_size = 16
########################################################

infer_MX(gen, save_dir, source_path, source_ext, gen_chars, ref_dict, load_img, batch_size)

{}